In [289]:
# Imports

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import logging
import numpy as np
import os
import math
import time

import tensorflow as tf
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [290]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [306]:
# Hyperparameters
nbEpochs = 50
train_batch_size = 100
test_batch_size = 100
dropoutChance = 0.5
lossControlPenalty = 0.4 # controlPenalty = 0.4...0.7
source = "SVHN"
target = "MNIST"
sourceChannels = 1
dataPath = os.path.dirname(os.getcwd()) + "/data/"
randomSeed = 1905
print(dataPath)

/home/pmirallesr/eclipse-workspace/DRCN-Torch/data/


In [307]:
np.random.seed(randomSeed)
torch.manual_seed(randomSeed)

**TEXT**

In [308]:
class GaussianDenoising:
    """Distort a pixel with additive [+ N(0,scale)] or multiplicative [x N(1,scale)] gaussian noise"""

    def __init__(self, sigma = 0.2, effectType = "additive"):
        self.sigma = sigma
        self.effectType = effectType

    def __call__(self, x):
        if effectType == "multiplicative":
            return x.numpy() * np.random.normal(loc = 1.0, scale = sigma, size = x.shape)
        elif effectType == "additive":
            return x.numpy + np.random.normal(loc = 0.0, scale = sigma, size = x.shape)
        else:
            print("Specify a valid type of gaussian error: multiplicative or additive")
            raise ValueError

In [309]:
class ImpulseDenoising:
    """Erase a pixel with probability p"""

    def __init__(self, p = 0.5):
        self.p = p

    def __call__(self, x):
        return x.numpy() * np.random.binomial(1, self.p, size=x.shape)

In [310]:
# Data Transforms
# Normalizations commented as torch already imports normalized data on 32x32

# Data Augmentation - Geometric Transformations
# 20º random rotation
# 20% random height and width shifts
dataAugmentTransforms = []
dataAugmentTransforms.append(torchvision.transforms.RandomAffine(degrees = 20, translate = (0.2, 0.2)))
# Denoising
dataAugmentTransforms.append(ImpulseDenoising())

MNIST_Transforms = []
MNIST_Transforms.append(torchvision.transforms.ToTensor())
#MNIST_Transforms.append(torchvision.transforms.Resize((32,32)))
# MNIST_Mean = 0.1307
# MNIST_StDev = 0.3081
# MNIST_Transforms.append(torchvision.transforms.Normalize(MNIST_Mean, MNIST_StDev))



SVHN_Transforms = []
SVHN_Transforms.append(torchvision.transforms.Grayscale())
SVHN_Transforms.append(torchvision.transforms.ToTensor())
# SVHN_Mean = 0.4657
# SVHN_StDev = 0.2025
# SVHN_Transforms.append(torchvision.transforms.Normalize(SVHN_Mean, SVHN_StDev))



In [311]:
train_MNIST_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.MNIST(dataPath, \
                train = True, download = True, \
                transform = torchvision.transforms.Compose\
                (MNIST_Transforms + dataAugmentTransforms)), batch_size = train_batch_size, shuffle = True)

test_MNIST_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.MNIST(dataPath, \
                train = False, download = True, \
                transform = torchvision.transforms.Compose\
                (MNIST_Transforms)), batch_size = test_batch_size, shuffle = True)

#DataLoader has irregular behaviour, does not autom create an SVHN folder but does so for MNIST
train_SVHN_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.SVHN(dataPath + "SVHN/", \
                split = "train", download = True, \
                transform = torchvision.transforms.Compose\
                (SVHN_Transforms)), batch_size = train_batch_size, shuffle = True)

test_SVHN_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.SVHN(dataPath + "SVHN/", \
                split = "test", download = True , \
                 transform = torchvision.transforms.Compose \
                (SVHN_Transforms + dataAugmentTransforms)), batch_size = test_batch_size, shuffle = True)


Using downloaded and verified file: /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/SVHN/train_32x32.mat
Using downloaded and verified file: /home/pmirallesr/eclipse-workspace/DRCN-Torch/data/SVHN/test_32x32.mat


In [312]:
def calcConvOutputDimensions(inputDim, kernelSize, padding = (0,0), dilation = (1,1), stride = (1,1)):
    variables = [inputDim, kernelSize, padding, dilation, stride]
    for i, elem in enumerate(variables):
        if isinstance(elem, int):
            variables[i] = (elem, elem)
        elif isinstance(elem, tuple):
            pass
        else:
            raise ValueError
    Hout = math.floor((variables[0][0] + 2*variables[2][0] - variables[3][0]*(variables[1][0] - 1) -1)/variables[4][0] + 1)
    Wout = math.floor((variables[0][1] + 2*variables[2][1] - variables[3][1]*(variables[1][1] - 1) -1)/variables[4][1] + 1)
    return (Hout, Wout)

def calcPoolOutputDimensions(inputDim, pooling):
    variables = [inputDim, pooling]
    for i, elem in enumerate(variables):
        if isinstance(elem, int):
            variables[i] = (elem, elem)
        elif isinstance(elem, tuple):
            pass
        else:
            raise ValueError
    return tuple(int(inputDim[i]/pooling[i]) for i,ti in enumerate(pooling) )
    

In [313]:
# Models
class Encoder(nn.Module):
    """Encoder common to Autoencoder and labeller"""

    def __init__(self, inputChannels, dropoutChance = 0.5):
        """Initialize DomainRegressor."""
        super(Encoder, self).__init__()
        
        #Size Parameters
        
        conv1Filters = 100
        conv1KernelSize = 5
        
        maxPool1Size = (2,2)
        
        conv2Filters = 150
        conv2KernelSize = 5
        
        maxPool2Size = (2,2)
        
        conv3Filters = 200
        conv3KernelSize = 3
        
        # Placeholder ranges
#         fc4OutputSize = range(300,1000,50)
#         fc5OutputSize = range(300,1000,50)
        fc4OutputDim = 300
        fc5OutputDim = 300
        
        
        # Convolutional Layers Size Calculations
        conv1InputChannels = inputChannels
        conv2InputChannels = conv1Filters
        conv3InputChannels = conv2Filters
                
        # Convolutional Layers
        self.conv1 = nn.Conv2d(conv1InputChannels, conv1Filters, conv1KernelSize)
        self.maxPool2D1 = nn.MaxPool2d(maxPool1Size)
        self.conv2 = nn.Conv2d(conv2InputChannels, conv2Filters, conv2KernelSize)
        self.maxPool2D2 = nn.MaxPool2d(maxPool2Size)
        self.conv3 = nn.Conv2d(conv3InputChannels, conv3Filters, conv3KernelSize)
        
        # Fully Connected Layers Size Calculations
        conv1OutputDim = calcConvOutputDimensions(32, conv1KernelSize) #Size of SVHN images!
        conv2InputDim = calcPoolOutputDimensions(conv1OutputDim, maxPool1Size)
        conv2OutputDim = calcConvOutputDimensions(conv2InputDim, conv2KernelSize)
        conv3InputDim = calcPoolOutputDimensions(conv2OutputDim, maxPool2Size)
        conv3OutputDim = calcConvOutputDimensions(conv3InputDim, conv3KernelSize)
        fc4InputDim = conv3Filters*conv3OutputDim[0]*conv3OutputDim[1]
        fc5InputDim = fc4OutputDim
        
        # Fully connected Layers
        self.fc4 = nn.Linear(fc4InputDim, fc4OutputDim)
        self.dropout4 = nn.Dropout(p = dropoutChance)
        
        self.fc5 = nn.Linear(fc5InputDim, fc5OutputDim)
        self.dropout5 = nn.Dropout(p = dropoutChance)
        
        

    def forward(self, x):
        """Forward pass X and return probabilities of source and domain."""
        x = F.relu(self.conv1(x))
        x = self.maxPool2D1(x)
        
        x = F.relu(self.conv2(x))
        x = self.maxPool2D2(x)
        
        x = F.relu(self.conv3(x))
        x = x.reshape(x.size(0), -1)
        
        x = F.relu(self.fc4(x))
        x = self.dropout4(x)
        x = F.relu(self.fc5(x))
        x = self.dropout5(x)
        return x

In [314]:
class Labeller(nn.Module):
    """ The labeller part of the network is constituted by 
    the common Encoder plus a labelling fully connected layer"""
    def __init__(self, encoder):
        super(Labeller, self).__init__()
        self.encoder = encoder
        fc3OutputSize = 10 # for 10 possible digits
        fc3InputSize = self.encoder.fc5.out_features
        self.fcOUT = nn.Linear(fc3InputSize, fc3OutputSize)  
        
    def forward(self, x):
        x = self.encoder(x)
        return F.relu(self.fcOUT(x))

    

In [315]:
class Autoencoder(nn.Module):
    """The autoencoder is constituted by the Encoder common to
    the labeller and itself, and a decoder part that is a mirror
    image of the Encoder
    
    Layers 6 and 7 are FC layers, layers 8 through 10 are (de)convolutional layers
    
    """

    def __init__(self, encoder):
        """Initialize DomainRegressor."""
        super(Autoencoder, self).__init__()
        
        self.encoder = encoder
        #Size Parameters
        
        deconv8InputSize = 100
        deconv8KernelSize = 5
        
        upscale8Size = self.encoder.maxPool2D2.kernel_size
        
        deconv9InputSize = 150
        deconv9KernelSize = 5
        
        upscale9Size = self.encoder.maxPool2D1.kernel_size
        
        deconv10InputSize = 200
        deconv10KernelSize = 3
        
        # Placeholder ranges - substitute for actual numbers for usage
#         fc7OutputSize = range(300,1000,50)
#         fc6OutputSize = range(300,1000,50)
        fc7OutputSize = 300
        fc6OutputSize = 300
        
        
        # Size Calculations
        deconv8OutputSize = self.encoder.conv3.in_channels
        deconv9OutputSize = self.encoder.conv2.in_channels
        deconv10OutputSize = self.encoder.conv1.in_channels
        
        #Is calcPoolSize still valid for Unspooling?
        
#         deconv9OutputSize = calcPoolOutputSize(deconv8OutputSize,maxPool8Size)
        
#         deconv10OutputSize = calcPoolOutputSize(deconv9OutputSize,maxPool9Size)
        
        deconv8InputSize = fc7OutputSize
        fc6InputSize = self.encoder.fc5.out_features
        fc7InputSize = fc6OutputSize
        
        # Layers
        
        self.fc7 = nn.Linear(fc7InputSize, fc7OutputSize)
        self.fc6 = nn.Linear(fc6InputSize, fc6OutputSize)
        
        self.deconv8 = nn.Conv2d(deconv8InputSize, deconv8OutputSize, deconv8KernelSize)
        self.upsample8 = nn.Upsample(scale_factor = 2, mode = "nearest")
        self.deconv9 = nn.Conv2d(deconv9InputSize, deconv9OutputSize, deconv9KernelSize)
        self.upsample9 = nn.Upsample(scale_factor = 2, mode = "nearest")
        self.deconv10 = nn.Conv2d(deconv10InputSize, deconv10OutputSize, deconv10KernelSize)
        
        


    def forward(self, x):
        """Forward pass X and return probabilities of source and domain."""
        x = self.encoder(x)
        
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        
        x = F.relu(self.deconv8(x))
        x = F.relu(self.maxUnspool2D8(x))
        
        x = F.relu(self.deconv9(x))
        x = F.relu(self.maxUnspool2D9(x))
        
        x = self.deconv10(x)
        
       
        return x

In [316]:
encoder = Encoder(sourceChannels)
labeller = Labeller(encoder)
autoencoder = Autoencoder(encoder)

In [319]:
optimizer = torch.optim.RMSprop(list(labeller.parameters()) + list(autoencoder.parameters()), lr=1e-4, alpha=0.9, eps=1e-08, weight_decay=0.9, momentum=0, centered=False)

In [ ]:
encoder.train()
labeller.train()
optimizer.zero_grad()
start = time.time()
for epoch in range(1, nbEpochs + 1):
    epochStart = time.time()
    for batch_id, (data, target) in enumerate(train_SVHN_loader):
        forward = labeller(data)
        loss = nn.CrossEntropyLoss()
        loss = lossControlPenalty * loss(forward, target)
#         tf.summary.scalar('Labeller_Loss', loss.detach(), epoch)
        print("Source batch {}:{}. Loss = {:5f}".format(epoch, batch_id, loss))
        loss.backward()
        labellerOptimizer.step()
        
    for batch_id, (data, target) in enumerate(train_MNIST_loader):
        print("Target batch {}:{}. Loss = {:5f}".format(epoch, batch_id, loss))
        forward = autoencoder(data)
        loss = nn.MSELoss()
        loss = (1 - lossControlPenalty) * loss(forward, target)
#         tf.summary.scalar('Autoencoder_Loss', loss.detach(), epoch)
        loss.backward()
        autoencoderOptimizer.step()
    epochEnd = time.time()
    print("------------\nEpoch {} completed in {:5f}\n{} s elapsed\n--------------".format(epoch, epochEnd - epochStart, epochEnd - start))
    

Source batch 1:0. Loss = 0.920518
Source batch 1:1. Loss = 0.920098
Source batch 1:2. Loss = 0.921502
Source batch 1:3. Loss = 0.920841
Source batch 1:4. Loss = 0.921345
Source batch 1:5. Loss = 0.920630
Source batch 1:6. Loss = 0.921894
Source batch 1:7. Loss = 0.921284
Source batch 1:8. Loss = 0.920780
Source batch 1:9. Loss = 0.921992
Source batch 1:10. Loss = 0.921837
Source batch 1:11. Loss = 0.920582
Source batch 1:12. Loss = 0.921401
Source batch 1:13. Loss = 0.920938
Source batch 1:14. Loss = 0.920776
Source batch 1:15. Loss = 0.919518
Source batch 1:16. Loss = 0.920362
Source batch 1:17. Loss = 0.920582
Source batch 1:18. Loss = 0.920827
Source batch 1:19. Loss = 0.922265
Source batch 1:20. Loss = 0.922041
Source batch 1:21. Loss = 0.920888
Source batch 1:22. Loss = 0.920321
Source batch 1:23. Loss = 0.922328
Source batch 1:24. Loss = 0.921375
Source batch 1:25. Loss = 0.920510
Source batch 1:26. Loss = 0.920531
Source batch 1:27. Loss = 0.921483
Source batch 1:28. Loss = 0.92